In [2]:
library(coin)
library(lme4)
library(lmerTest)
library(plyr)
library(xtable)
library(LMERConvenienceFunctions)
library(car)

Loading required package: survival
Loading required package: Matrix

Attaching package: 'lmerTest'

The following object is masked from 'package:lme4':

    lmer

The following object is masked from 'package:stats':

    step

Loading required package: carData
Registered S3 methods overwritten by 'car':
  method                          from
  influence.merMod                lme4
  cooks.distance.influence.merMod lme4
  dfbeta.influence.merMod         lme4
  dfbetas.influence.merMod        lme4


In [3]:
get_processed_data <- function(data_path){
    data <- read.table(file.path(data_path, "measures.csv"), header = TRUE, sep = ",")
    data$subj_id <- as.factor(data$subj_id)
    
    data$tta_condition <- scale(data$tta_condition)
    data$d_condition <- scale(data$d_condition)
    data$v_condition <- scale(data$v_condition)
    data$RT <- scale(data$RT)
    
    data$decision <- relevel(data$decision, ref=2)
  
    return(data)
}

data <- get_processed_data("../data")

In [4]:
contrasts(data$decision) 

,Go
Stay,0
Go,1


In [5]:
data

subj_id,session,route,intersection_no,idx_bot_spawn,idx_response,idx_min_distance,min_distance,RT,tta_condition,d_condition,v_condition,is_turn_decision,decision
<fct>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,"<dbl[,1]>","<dbl[,1]>","<dbl[,1]>","<dbl[,1]>",<fct>,<fct>
129,1,1,5,114,161,471,3.226016,-0.7051107,-1.216967155,-1.20925732,-0.32457895,False,Stay
129,1,1,6,41,56,389,16.099580,-0.9176350,0.005423809,-1.20925732,-1.01351846,True,Go
129,1,1,8,45,87,506,3.234011,-0.7373682,0.005423809,-1.20925732,-1.01351846,False,Stay
129,1,1,10,77,124,430,8.046337,-0.7053050,-1.216967155,1.23364570,1.97188609,True,Go
129,1,1,11,84,92,380,48.915927,-0.9647906,1.227814773,0.01219419,-0.70732312,True,Go
129,1,1,13,49,111,414,33.930989,-0.6050345,1.227814773,0.01219419,-0.70732312,True,Go
129,1,1,14,0,68,393,3.625540,-0.5274997,-1.216967155,1.23364570,1.97188609,False,Stay
129,1,1,15,57,102,403,7.893319,-0.7186485,-1.216967155,0.01219419,0.82365357,True,Go
129,1,1,19,59,125,459,12.732931,-0.5784770,0.005423809,0.01219419,-0.09493244,True,Go


# Decision as a function of TTA and d

In [6]:
rnd_effects_analysis_decision <- function(data){
    rnd1 = glmer(decision ~ tta_condition + d_condition + (1|subj_id), data, family = binomial)
    rnd2 = glmer(decision ~ tta_condition + d_condition + (d_condition|subj_id), data, family = binomial)
    rnd3 = glmer(decision ~ tta_condition + d_condition + (tta_condition|subj_id), data, family = binomial)
    rnd4 = glmer(decision ~ tta_condition + d_condition + (d_condition + tta_condition|subj_id), data, family = binomial)
    
    rnd.anova = anova(rnd1, rnd2, rnd3, rnd4)
    print(rnd.anova)
    
    print("Best model according to AIC")
    print(row.names(rnd.anova[rnd.anova$AIC==min(rnd.anova$AIC), ]))
    print("Best model according to BIC")
    print(row.names(rnd.anova[rnd.anova$BIC==min(rnd.anova$BIC), ]))
}

In [10]:
rnd_effects_analysis_decision(data)

Data: data
Models:
rnd1: decision ~ tta_condition + d_condition + (1 | subj_id)
rnd2: decision ~ tta_condition + d_condition + (d_condition | subj_id)
rnd3: decision ~ tta_condition + d_condition + (tta_condition | subj_id)
rnd4: decision ~ tta_condition + d_condition + (d_condition + tta_condition | 
rnd4:     subj_id)
     Df    AIC    BIC  logLik deviance  Chisq Chi Df Pr(>Chisq)    
rnd1  4 1782.6 1804.6 -887.30   1774.6                             
rnd2  6 1754.3 1787.3 -871.15   1742.3 32.301      2  9.679e-08 ***
rnd3  6 1759.3 1792.3 -873.64   1747.3  0.000      0          1    
rnd4  9 1739.3 1788.8 -860.68   1721.3 25.924      3  9.896e-06 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
[1] "Best model according to AIC"
[1] "rnd4"
[1] "Best model according to BIC"
[1] "rnd2"


In [16]:
fitted_model = glmer(decision ~ tta_condition + d_condition + (d_condition + tta_condition | subj_id), data, family = binomial)
print(summary(fitted_model))

output = summary(fitted_model)$coefficients

file_name = file.path("output", "tab_decision.tex")
print(xtable(output, display = c("g","g","g","g","g"), # digits = c(2,3,2,2,2,-2), 
             label = "tab:decision", 
             caption = "Results of statistical analysis of the effect of distance and TTA conditions on decision. The mixed-effects model included random slopes of distance and TTA conditions per participant."), 
      caption.placement = "top", table.placement="!h", math.style.exponents = TRUE, floating.environment = "table*", 
      type = "latex", file = file_name, booktabs = TRUE)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
decision ~ tta_condition + d_condition + (d_condition + tta_condition |  
    subj_id)
   Data: data

     AIC      BIC   logLik deviance df.resid 
  1739.4   1788.8   -860.7   1721.4     1794 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.9117 -0.5081 -0.0903  0.5635  5.7029 

Random effects:
 Groups  Name          Variance Std.Dev. Corr       
 subj_id (Intercept)   1.2583   1.1217              
         d_condition   0.1928   0.4391   -0.82      
         tta_condition 0.1657   0.4070   -0.11  0.18
Number of obs: 1803, groups:  subj_id, 16

Fixed effects:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept)    -0.2916     0.2896  -1.007    0.314    
tta_condition   1.0227     0.1235   8.279   <2e-16 ***
d_condition     1.2201     0.1327   9.192   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' '

In [17]:
coef(fitted_model)

$subj_id
    (Intercept) tta_condition d_condition
129 -0.22961688     1.2200793   1.1068231
280 -1.44429448     1.3054526   1.6908883
389 -0.96813288     1.1432608   1.6233692
421 -0.11225190     0.5783181   1.0780586
525  0.35795749     0.9984548   0.8657134
616 -1.60498388     1.0798815   1.6280227
618 -0.46906774     1.2049597   1.3088798
642 -0.93167896     0.5160703   1.4909221
652 -1.20482758     1.3478159   1.1815854
755 -0.02945055     0.6863434   1.3599629
776  0.90472479     1.0718537   1.1146721
827  1.18174029     1.2512319   0.8144491
853 -2.54321587     0.9266159   1.8778666
913  0.68079208     1.4629522   0.7846116
980  1.04522355     0.1924725   0.4655050
996  0.91449363     1.0818281   0.8406018

attr(,"class")
[1] "coef.mer"

# RT as a function of TTA and distance

In [10]:
rnd_effects_analysis_RT <- function(data){
    rnd1 = lmer(RT ~ tta_condition + d_condition + (1|subj_id), data)
    rnd2 = lmer(RT ~ tta_condition + d_condition + (d_condition|subj_id), data)
    rnd3 = lmer(RT ~ tta_condition + d_condition + (tta_condition|subj_id), data)
    rnd4 = lmer(RT ~ tta_condition + d_condition + (d_condition + tta_condition|subj_id), data)
    
    rnd.anova = anova(rnd1, rnd2, rnd3, rnd4)
    print(rnd.anova)
    
    print("Best model according to AIC")
    print(row.names(rnd.anova[rnd.anova$AIC==min(rnd.anova$AIC), ]))
    print("Best model according to BIC")
    print(row.names(rnd.anova[rnd.anova$BIC==min(rnd.anova$BIC), ]))
}

In [16]:
rnd_effects_analysis_RT(data[data$decision=="Go",])

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model failed to converge with max|grad| = 0.00337505 (tol = 0.002, component 1)"boundary (singular) fit: see ?isSingular
boundary (singular) fit: see ?isSingular
refitting model(s) with ML (instead of REML)


Data: data
Models:
rnd1: RT ~ tta_condition + d_condition + (1 | subj_id)
rnd2: RT ~ tta_condition + d_condition + (d_condition | subj_id)
rnd3: RT ~ tta_condition + d_condition + (tta_condition | subj_id)
rnd4: RT ~ tta_condition + d_condition + (d_condition + tta_condition | 
rnd4:     subj_id)
     Df     AIC     BIC logLik deviance  Chisq Chi Df Pr(>Chisq)  
rnd1  5 -930.77 -907.00 470.38  -940.77                           
rnd2  7 -935.09 -901.80 474.54  -949.09 8.3180      2    0.01562 *
rnd3  7 -933.03 -899.75 473.52  -947.03 0.0000      0    1.00000  
rnd4 10 -934.01 -886.46 477.00  -954.01 6.9747      3    0.07271 .
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
[1] "Best model according to AIC"
[1] "rnd2"
[1] "Best model according to BIC"
[1] "rnd1"


Model with rnd2 fails to converge, so using random intercept only

In [13]:
fitted_model = lmer(RT ~ tta_condition + d_condition + (1 | subj_id), data[data$decision=="Go",])
print(summary(fitted_model))

output = summary(fitted_model)$coefficients

file_name = file.path("output", "tab_RT.tex")
print(xtable(output, display = c("g","g","g","g","g","g"), # digits = c(2,3,2,2,2,-2), 
             label = "tab:RT", 
             caption = "Results of statistical analysis of the effect of distance and TTA conditions on response time in ``go'' decisions. The mixed-effects model included random intercept per participant."), 
      caption.placement = "top", table.placement="!h", math.style.exponents = TRUE, floating.environment = "table*", 
      type = "latex", file = file_name, booktabs = TRUE)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: RT ~ tta_condition + d_condition + (1 | subj_id)
   Data: data[data$decision == "Go", ]

REML criterion at convergence: -917.3

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.6444 -0.6198 -0.1025  0.4356  5.5998 

Random effects:
 Groups   Name        Variance Std.Dev.
 subj_id  (Intercept) 0.005702 0.07551 
 Residual             0.018633 0.13650 
Number of obs: 858, groups:  subj_id, 16

Fixed effects:
                Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)    -0.618564   0.019830  15.424602 -31.194 2.34e-15 ***
tta_condition   0.023189   0.005127 845.215875   4.523 6.97e-06 ***
d_condition    -0.005384   0.005244 849.995403  -1.027    0.305    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Correlation of Fixed Effects:
            (Intr) tt_cnd
tta_conditn -0.116       
d_condition -0.138  0.126


In [14]:
coef(fitted_model)

$subj_id
    (Intercept) tta_condition  d_condition
129  -0.4957254    0.02318913 -0.005384352
280  -0.5324027    0.02318913 -0.005384352
389  -0.6168761    0.02318913 -0.005384352
421  -0.6647730    0.02318913 -0.005384352
525  -0.7534204    0.02318913 -0.005384352
616  -0.6027528    0.02318913 -0.005384352
618  -0.6126258    0.02318913 -0.005384352
642  -0.5895560    0.02318913 -0.005384352
652  -0.4611511    0.02318913 -0.005384352
755  -0.6614222    0.02318913 -0.005384352
776  -0.6621704    0.02318913 -0.005384352
827  -0.6543934    0.02318913 -0.005384352
853  -0.6047982    0.02318913 -0.005384352
913  -0.6655934    0.02318913 -0.005384352
980  -0.6491865    0.02318913 -0.005384352
996  -0.6701841    0.02318913 -0.005384352

attr(,"class")
[1] "coef.mer"